In [2]:
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
import os


In [3]:
pwd=os.environ['PGPASS']
uid=os.environ['PGUID']

driver="{SQL Server Native Client 11.0}"
server='LAPTOP-K6D5BFPK'
database="AdventureWorks2019;"

def extract():
    try:
        src_conn=pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + '\SQLEXPRESS' + ';DATABASE=' + database + ';UID=' + uid + ';PWD=' + pwd )
        src_cursor=src_conn.cursor()
        print('hello')
#         src_cursor.execute(""" select t.name as table_name from sys.tables t where t.name in ('Product','ProductSubcategory' ) """)
#         src_tables=src_cursor.fetchall()
        query = """ select  t.name as table_name
        from sys.tables t where t.name in ('Product','ProductSubcategory') """
        src_tables = pd.read_sql_query(query, src_conn).to_dict()['table_name']
        print(src_tables)
        
        print('bolo')
#         for tbl in src_tables:
#             print('hiiiiiii')
#             df=pd.read_sql_query(f'select * FROM {tbl[0]}', src_conn)
#             load(df, tbl[0])
        for id in src_tables:
            table_name = src_tables[id]
            df = pd.read_sql_query(f'select * FROM {table_name}', src_conn)
            load(df, table_name)

    except Exception as e:
        print("Data extract error: "+str(e))
    finally:
        src_conn.close()

def load(df,tbl):
    print('hey')
    try:
        rows_imported=0
        engine=create_engine(f'postgresql://{uid}:{pwd}@{server}:5433/AdventureWorks')
        printf(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')

        df.to_sql(f'stg_{tbl}',engine, if_exists='replace', index=False)
        rows_imported+=len(df)
        print("Data imported successful")
    except Exception as e:
        print("Data load error: "+str(e))


In [ ]:
extract()